### Algorithm to Approximate Stabilizer Rank
* https://arxiv.org/abs/1711.07848 - On the Geometry of Stabilizer States
* $\underline{\text{First Approach - Approximating an arbitrary quantum state with a single stabilizer state (Section 4.1)}}$
    * Outline of approach:
        * Begin with arbitrary state - 
        $$\ket{\psi} = \sum_{i = 1}^{k} \alpha_i \ket{s_i}$$
        * Start at stabilizer state $\ket{s_m}$ such that it has largest $|\alpha_i|$
        * At each iteration, evaluate $N(\ket{s_m})$ (nearest neighbors of $\ket{s_m}$) by computing 
        $$\max_{\ket{\phi} \in N(\ket{s_m})} |\bra{\phi}\ket{\psi}|$$
* $\underline{\text{Second Approach - Approximating arbitrary states with superpositions of stabilizer states (Section 4.2)}}$
    * 

In [2]:
import stim
import numpy as np
from typing import List, Tuple

In [29]:
%run -i NonCliff_1Meas.ipynb

[(0.7071067811865475-0j)_Z] + [(-0.7071067811865475+0j)_Y]
The minimum eigenvalue for this pauli sum is: (0.9999999999999996+0j)
The associated eigenvector is: [ 0.92387953+0.j         -0.        -0.38268343j -0.        -0.j
 -0.        -0.j        ]


In [15]:
def get_orthogonal_vecs(used_neighbors: np.array, neighbor_list: List) -> List:
    """ 
    Finds the set of vectors neighboring vectors that are orthogonal to 
    vectors that have already been used as optimal neighbors

    Parameters:
    -----------
    used_neighbors - List of previous optimal choice of nearest neighbors
    neighbor_list - List of nearest neighbors

    Returns:
    --------
    List of possible options for next optimal neighbor
    """
    option_list = []
    if (used_neighbors == []):
        option_list = neighbor_list 
    else:
        option_list = [neighbor for neighbor in neighbor_list for used in used_neighbors if (np.abs(np.vdot(used, neighbor)) == 0)]
    return option_list

def state_max_coeff(eig_vec: np.array, basis_vecs: List) -> int:
    """ 
    Finds the constituent stabilizer state (computational basis state) with 
    largest coefficient amplitude

    Parameters:
    -----------
    eig_vec - Eigenvector 
    basis_vecs - List of basis vectors

    Returns:
    --------
    Number corresponding to basis vector with largest coefficient amplitude
    """
    max_pos = np.argmax(np.abs(eig_vec))
    return max_pos

def comp_base_vecs(num_qubits: int) -> List:
    """
    Return all computational basis vectors for a given number of qubits

    Parameters:
    -----------
    num_qubits - Number of qubits

    Returns:
    --------
    Basis vectors in list
    """
    basis_vecs = []
    for i in range(2**num_qubits):
        start_vec = np.zeros((2**num_qubits,), dtype=complex)
        start_vec[i] = 1+0j
        basis_vecs.append(start_vec)
    
    return basis_vecs

def find_nearest_neighbors(stab_state: np.array, basis_vecs: List) -> List:
    """ 
    Find and return all nearest neighbor states to chosen stabilizer state 'stab_state'

    Parameters:
    -----------
    stab_state - Stabilizer state for which we wish to find nearest neighbors
    basis_vecs - Computational Basis vectors

    Returns:
    --------
    List of nearest neighbor states
    """
    phases = [1, -1, 1j, -1j]
    norm_factor = np.sqrt(2)
    nearest_neighbors = []
    for p in phases:
        for vec in basis_vecs:
            if (np.array_equal(stab_state, vec)):
                continue
            else:
                neighbor_state = np.add(stab_state, p * vec)/norm_factor 
                nearest_neighbors.append(neighbor_state)
    
    nearest_neighbors.append(stab_state)
    return nearest_neighbors

def find_nearest_neighbors_general(stab_state: np.array) -> List:
    """ 
    Find and return all nearest neighbor states for chosen stabilizer state 'stab_state' for 
    states other than computational basis states

    Parameters:
    -----------
    stab_state - Stabilizer state for which we wish to find nearest neighbors

    Returns:
    --------
    List of nearest neighbor states
    """
    phases = [1, -1, 1j, -1j]
    norm_factor = np.sqrt(2)
    nearest_neighbors = []
    

    

def optimal_nearest_neighbor(eig_vec: np.array, nearest_neighbors: List, chosen_neighbors: List) -> np.array:
    """ 
    Find nearest neighbor stabilizer state with maximum inner product

    Parameters:
    -----------
    eig_vec - Eigenvector we are trying to approximate
    nearest_neighbors - Nearest neighbors list
    chosen_neighbors - optimal neighbors chosen in previous iterations

    Returns:
    --------
    Single stabilizer state that is closest to 'eig_vec'
    """
    neighbor_options = get_orthogonal_vecs(chosen_neighbors, nearest_neighbors)
    #neighbor_options = nearest_neighbors
    return nearest_neighbors[np.argmax([np.abs(np.vdot(eig_vec, neighbor)) for neighbor in neighbor_options])]

In [16]:
fin_stab_approx = np.zeros((2**total_qubits,), dtype=complex)
min_eigvec_cp = min_eigvec.copy()
basis_vecs = comp_base_vecs(total_qubits)
stab_state = basis_vecs[state_max_coeff(min_eigvec, basis_vecs)]
neighbor_list = find_nearest_neighbors(stab_state, basis_vecs)
optimal_neighbor = optimal_nearest_neighbor(min_eigvec, neighbor_list, [])
print(optimal_neighbor)
print(np.abs(np.vdot(optimal_neighbor, min_eigvec)))



[0.        -0.70710678j 0.70710678+0.j        ]
0.9238795325112868


In [8]:
def stab_rank_approx_one(num_qubits: int, eig_vec: np.array)  -> int:
    """ 
    Approximate the stabilizer rank (using the first approach)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0
    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    used_neighbors = []
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    final_stab_approx = np.zeros((2**num_qubits,), dtype=complex) # Keep track of the final approximation
    for _ in range(2**num_qubits):
        print("The eigenvector currently looks like: " + str(eig_vec))

        max_coeff = state_max_coeff(eig_vec, basis_vecs)
        #used_basis_vecs.append(max_coeff)
        stab_state = basis_vecs[max_coeff] # Figure out which stabilizer state to find neighbors for first
        neighbors = find_nearest_neighbors(stab_state, basis_vecs) # Find all nearest neighbors of 'stab_state'
        optimal_neighbor = optimal_nearest_neighbor(eig_vec, neighbors, used_neighbors) # Find optimal nearest neighbor of 'stab_state'
        used_neighbors.append(optimal_neighbor)
        print("The optimal neighbor for the current state is: " + str(optimal_neighbor))

        # Subtract optimal neighbor component from eigenvector
        # Then normalize state
        eig_vec = np.subtract(eig_vec, optimal_neighbor)
        eig_vec = eig_vec/np.linalg.norm(eig_vec)

        # Add to optimal neighbor component to 'final_stab_approx'
        # Then normalize state 
        final_stab_approx = np.add(final_stab_approx, optimal_neighbor)
        final_stab_approx = final_stab_approx/np.linalg.norm(final_stab_approx)
        print("The approximated stabilizer state so far is: " + str(final_stab_approx))

        # Check to see how inner product between 'final_stab_approx' and 
        # 'eig_vec' progresses 
        print("The inner product between approximation and actual is: " + str(np.abs(np.vdot(final_stab_approx, init_state))))
        print("\n")
        approx_stab_rank += 1

    return approx_stab_rank

In [23]:
def stab_rank_approx_two(num_qubits: int, eig_vec: np.array) -> int:
    """ 
    Approximate the stabilizer rank (2nd attempt)

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector for which we'd like to evaluate stabilizer rank

    Returns:
    --------
    Approximate stabilizer rank for eigenvector
    """
    approx_stab_rank = 0
    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    used_neighbors = []
    init_state = eig_vec.copy() # Make a copy of the original eigenvector
    final_stab_approx = np.zeros((2**num_qubits,), dtype=complex) # Keep track of the final approximation
    for _ in range(2**num_qubits):
        print("The eigenvector currently looks like: " + str(eig_vec))
        #print("The current norm of the eigenvector is: " + str(np.linalg.norm(eig_vec)))

        # Find basis state with maximum coefficient
        max_coeff = state_max_coeff(eig_vec, basis_vecs)
        stab_state = basis_vecs[max_coeff]

        # Compute neighbors for 'stab_state' and retrieve optimal neighbor
        neighbors = find_nearest_neighbors(stab_state, basis_vecs)
        optimal_neighbor = optimal_nearest_neighbor(eig_vec, neighbors, used_neighbors)
        used_neighbors.append(optimal_neighbor)
        print("The optimal neighbor for the current state is: " + str(optimal_neighbor))

        # Subtract optimal neighbor from eigenvector 
        eig_vec = np.subtract(eig_vec, optimal_neighbor)

        # Add to optimal neighbor component to 'final_stab_approx'
        # Then normalize state 
        final_stab_approx = np.add(final_stab_approx, optimal_neighbor)
        final_stab_approx = final_stab_approx/np.linalg.norm(final_stab_approx)
        print("The approximated stabilizer state so far is: " + str(final_stab_approx))

        # Check to see how inner product between 'final_stab_approx' and 
        # 'eig_vec' progresses 
        print("The inner product between approximation and actual is: " + str(np.abs(np.vdot(final_stab_approx, init_state))))
        print("\n")
        approx_stab_rank += 1

    return approx_stab_rank

In [27]:
def one_state_state_approx(num_qubits: int, eig_vec: np.array) -> np.array:
    """ 
    Returns the best single stabilizer state approximation for 'eig_vec' 

    Parameters:
    -----------
    num_qubits - Number of qubits
    eig_vec - Eigenvector we wish to approximate

    Returns:
    --------
    Best one state approximation for our eigenvector
    """
    threshold = 0.01 
    basis_vecs = comp_base_vecs(num_qubits) # Basis vectors given number of qubits
    used_neighbors = []
    init_state = eig_vec.copy() # Make a copy of the original eigenvector

    


    

In [30]:
#approx_stab_rank = stab_rank_approx_one(total_qubits, min_eigvec)
#approx_stab_rank = stab_rank_approx_two(total_qubits, min_eigvec)
approx_state = one_state_state_approx(total_qubits, min_eigvec)

The eigenvector is: [ 0.92387953+0.j         -0.        -0.38268343j -0.        -0.j
 -0.        -0.j        ]
The current eigenvector approximation is: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
The absolute value of inner product between approx and actual so far is: 0.9238795325112867


The eigenvector is: [ 0.92387953+0.j         -0.        -0.38268343j -0.        -0.j
 -0.        -0.j        ]


ValueError: attempt to get argmax of an empty sequence

* Based on the above results, testing for various eigenvectors for Pauli Sums, we find that this algorithm gets its best approximation in the first iteration
* Past one iteration, the approximations move too far away from the true eigenvector